## 5-10 Fiber
### 5-10-1 基本的な振る舞い

In [2]:
fiber = Fiber.new {
  puts 'Hello, Fiber'
  Fiber.yield
  puts 'Hello (again)'
}

fiber.resume

fiber.resume

fiber.resume

Hello, Fiber
Hello (again)


FiberError: dead fiber called

### 5-10-2 引数と戻り値

In [3]:
fiber = Fiber.new { |first|
  puts first
  
  second = Fiber.yield('goodbye')
  
  puts second
  
  'goodbye (again)'
}

puts fiber.resume('hello')
puts fiber.resume('hello (again)')

hello
goodbye
hello (again)
goodbye (again)


### 5-10-3 ジェネレータ

In [4]:
fib = Fiber.new {
  a, b = 0, 1
  
  loop do
    a, b = b, a+b
    
    Fiber.yield(a)
  end
}

puts 10.times.map { fib.resume }
puts fib.resume
puts fib.resume

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
89
144


In [5]:
fib.resume

233

- `Proc` との差異は？
- @kishima: `Fiber` はmrubyのコンパイル時に並列で処理するのに使っている。並列処理はOSでやって、待ちをFiberでやっている。なお、Cygwinではこのビルドが壊れている。

### 5-10-4 スレッドと組み合わせた場合の制限

In [6]:
fiber = Fiber.new {
  puts 'OMG'
}

Thread.fork {
  fiber.resume
}.join

#<Thread:0x00007fa5898e4ff0@<main>:4 run> terminated with exception (report_on_exception is true):
<main>:5:in `resume': fiber called across threads (FiberError)
	from <main>:5:in `block in <main>'


FiberError: fiber called across threads

In [7]:
fiber = Fiber.new {
  Thread.fork {
    Fiber.yield
  }.join
}

fiber.resume

#<Thread:0x00007fa58a91e330@<main>:1 run> terminated with exception (report_on_exception is true):
<main>:2:in `yield': can't yield from root fiber (FiberError)
	from <main>:2:in `block (2 levels) in <main>'


FiberError: can't yield from root fiber

次回は、「5-11 Process」から